# Pandera

Pandera provides a flexible and expressive API for performing data validation on dataframes to make data processing pipelines more readable and robust. Dataframes contain information that pandera explicitly validates at runtime. This is useful in production-critical data pipelines or reproducible research settings.

In [37]:
# Importando o Pandas e Pandera
import pandas as pd
import pandera as pa
# Omitir alguns warnings
pd.options.mode.chained_assignment = None

In [24]:
# Criando dataframes com base nas tabelas
df_certo = pd.read_excel("exemplo_validacao_certo.xlsx", usecols = "A:K")
df_errado = df_certo.copy(deep=True)

O `df_certo` é o df em que todas as validações devem passar. Já o `df_errado` é o df em que teremos problemas em alguns testes. 

Lista de validações que realizaremos:
1. Verificar se `id_compra` é único
2. Verificar se todas as datas em `data` está corretas
3. Verificar se `ano_mes_id` está coerente com `data`
4. Verificar se `ano` está coerente com `data`
5. Verificar se `mes` está coerente com `data`
6. Verificar se `dia` está coerente com `data`
7. Verificar se o valor `total` é a soma de `valor` e `frete`
8. Verificar se o `produto` faz parte do catálogo
9. Verificar se os tipos de cada coluna estão coerentes

In [57]:
# Definindo Schema para verificar validações
schema = pa.DataFrameSchema(
    checks = [pa.Check(lambda df : df["total"] == df["valor"] + df["frete"], name = "Valor total está errado")],
    columns = {
        "usuario": pa.Column(pa.String, allow_duplicates = True, required = True),
        "id_compra": pa.Column(pa.Int, allow_duplicates = False, required = True),
        "data": pa.Column(pa.DateTime, allow_duplicates=True, required = True),
        "ano_mes_id": pa.Column(pa.Int, allow_duplicates=True, required = True),
        "ano": pa.Column(pa.Int, allow_duplicates=True, required = True),
        "mes": pa.Column(pa.Int, allow_duplicates=True, required = True),
        "dia": pa.Column(pa.Int, allow_duplicates=True, required = True),
        "produto": pa.Column(pa.String, allow_duplicates=True, required = True),
        "valor": pa.Column(pa.Float, allow_duplicates=True, required = True),
        "frete": pa.Column(pa.Float, allow_duplicates=True, required = True),
        "total": pa.Column(pa.Float, allow_duplicates=True, required = True)
})

In [58]:
# Passando a validação de tipos pelo df_certo
schema.validate(df_certo).head()

,usuario,id_compra,data,ano_mes_id,ano,mes,dia,produto,valor,frete,total
0,Sergio,1,2022-01-12,202201,2022,1,12,capinha,4.463521,6.784318,11.247839
1,João,2,2022-01-26,202201,2022,1,26,mouse,1172.596978,10.877619,1183.474597
2,Paulo,3,2022-01-17,202201,2022,1,17,tablet,467.393957,8.865805,476.259762
3,Paulo,4,2022-01-26,202201,2022,1,26,pelicula,1001.749439,9.149199,1010.898638
4,Sandy,5,2022-01-12,202201,2022,1,12,cabo,1394.304542,6.959222,1401.263764


Vamos inserir erros no `df_errado` que no momento é uma cópia de `df_certo`.

In [59]:
# Duplicando um id_compra e validando
df_errado["id_compra"][1] = 1
try:
    schema.validate(df_errado).head()
except Exception as e:
    print(e)
# Resetando
df_errado = df_certo.copy()

series 'id_compra' contains duplicate values: {1: 1}


In [60]:
# Duplicando um id_compra e validando
df_errado["total"][15] = 0.00
try:
    schema.validate(df_errado).head()
except Exception as e:
    print(e)
# Resetando
df_errado = df_certo.copy()

<Schema DataFrameSchema(
    columns={
        'usuario': <Schema Column(name=usuario, type=DataType(str))>
        'id_compra': <Schema Column(name=id_compra, type=DataType(int64))>
        'data': <Schema Column(name=data, type=DataType(datetime64[ns]))>
        'ano_mes_id': <Schema Column(name=ano_mes_id, type=DataType(int64))>
        'ano': <Schema Column(name=ano, type=DataType(int64))>
        'mes': <Schema Column(name=mes, type=DataType(int64))>
        'dia': <Schema Column(name=dia, type=DataType(int64))>
        'produto': <Schema Column(name=produto, type=DataType(str))>
        'valor': <Schema Column(name=valor, type=DataType(float64))>
        'frete': <Schema Column(name=frete, type=DataType(float64))>
        'total': <Schema Column(name=total, type=DataType(float64))>
    },
    checks=[
        <Check Valor total está errado>
    ],
    coerce=False,
    dtype=None,
    index=None,
    strict=False
    name=None,
    ordered=False
)> failed element-wise validator 0